## RNN Play Generator

Now time for one of the coolest examples we've seen so far. We are going to use a RNN to generate a play. We will simply show the RNN an example of something we want it to recreate and it will learn how to write a version of it on its own. We'll do this using a character predictive model that will take as input a variable length sequence and predict the next character. We can use the model many times in a row with the output from the last predicition as the input for the next call to generate a sequence.


*This is based on the following: https://www.tensorflow.org/tutorials/text/text_generation*

shakespare text file: *https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt*

In [1]:
# imports
import keras
import tensorflow as tf
import os
import numpy as np

### Dataset
For this example, we only need one peice of training data. In fact, we can write our own poem or play and pass that to the network for training if we'd like. However, to make things easy we'll use an extract from a shakesphere play.

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


### Loading Your Own Data
To load your own data, you'll need to upload a file from the dialog below. Then you'll need to follow the steps from above but load in this new file instead.

In [ ]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

### Read Contents of File
Let's look at the contents of the file.

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



### Encoding
Since this text isn't encoded yet well need to do that ourselves. We are going to encode each unique character as a different integer.

In [5]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [6]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


And here we will make a function that can convert our numeric values to text.


In [7]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


### Creating Training Examples
Remember our task is to feed the model a sequence and have it return to us the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples. 

The training examples we will prepapre will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

```input: Hell | output: ello```

Our first step will be to create a stream of characters from our text data.

In [8]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

Next we can use the batch method to turn this stream of characters into batches of desired length.

In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

Now we need to use these sequences of length 101 and split them into input and output.

In [10]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [11]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


Finally we need to make training batches.

In [12]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

### Building the Model
Now it is time to build the model. We will use an embedding layer a LSTM and one dense layer that contains a node for each unique character in our training data. The dense layer will give us a probability distribution over all nodes.

In [13]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


### Creating a Loss Function
Now we are going to create our own loss function for this problem. This is because our model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch. 



However, before we do that let's have a look at a sample input and the output from our untrained model. This is so we can understand what the model is giving us.



In [14]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [15]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-2.2400492e-03 -3.4750407e-03  3.4641521e-03 ...  3.2507773e-03
   -6.4873421e-03  1.7617734e-03]
  [-9.6056554e-03 -7.1073282e-03 -2.4814699e-03 ... -1.8949769e-03
   -1.8139180e-03  4.6120323e-03]
  [-8.7340809e-03 -4.1179787e-03  2.0244115e-04 ... -6.1437860e-03
   -1.8808136e-03  9.8221761e-04]
  ...
  [-2.2781733e-03 -2.7634287e-03  6.9931373e-03 ...  3.1686106e-03
   -8.4274057e-03 -1.2225755e-03]
  [-1.0456568e-02 -6.8951738e-03  7.7411053e-03 ...  4.3723774e-03
   -6.8891067e-03 -3.0977107e-03]
  [-1.7279380e-03 -3.4068534e-03  8.0568539e-03 ...  5.7815714e-03
   -3.4643300e-03 -4.4840020e-03]]

 [[-2.2400492e-03 -3.4750407e-03  3.4641521e-03 ...  3.2507773e-03
   -6.4873421e-03  1.7617734e-03]
  [-6.5646847e-03 -6.4572054e-03  1.6799793e-03 ... -9.1607345e-04
   -3.0712388e-03  7.1817716e-03]
  [-4.1879850e-04 -6.0450211e-03  3.8529616e-03 ... -1.4737655e-03
   -1.5372834e-03  2.4336178e-03]
  ...
  [-2.0710392e-02 -6.4709191e-03 -2.3456642e-03 ... -5.5662729e

In [16]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00224005 -0.00347504  0.00346415 ...  0.00325078 -0.00648734
   0.00176177]
 [-0.00960566 -0.00710733 -0.00248147 ... -0.00189498 -0.00181392
   0.00461203]
 [-0.00873408 -0.00411798  0.00020244 ... -0.00614379 -0.00188081
   0.00098222]
 ...
 [-0.00227817 -0.00276343  0.00699314 ...  0.00316861 -0.00842741
  -0.00122258]
 [-0.01045657 -0.00689517  0.00774111 ...  0.00437238 -0.00688911
  -0.00309771]
 [-0.00172794 -0.00340685  0.00805685 ...  0.00578157 -0.00346433
  -0.004484  ]], shape=(100, 65), dtype=float32)


In [17]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-2.2400492e-03 -3.4750407e-03  3.4641521e-03 -7.2887810e-03
 -1.8879981e-04 -1.2620051e-03  1.9105009e-03  1.3025708e-03
 -1.6009748e-03  1.3192141e-03 -7.5248745e-04  1.7325627e-05
  5.8689504e-04 -8.9738681e-04 -2.8217712e-03 -5.5938433e-03
 -1.7569956e-03 -2.4945587e-03  1.5004971e-03  1.6827964e-03
 -7.0039304e-03  1.0960051e-03 -2.9868139e-03 -3.3735661e-03
  2.3053400e-03 -3.1011729e-03 -3.1738831e-03 -4.0329117e-03
 -1.8913526e-03  1.1219615e-02  4.7787344e-03 -1.0325852e-03
 -1.2350080e-03 -3.9371904e-03 -1.6156598e-03 -1.2605977e-03
  1.5218367e-03  5.2629453e-03 -2.2120506e-03 -2.1698114e-03
  4.8376615e-03 -2.6985996e-03  5.3846752e-03  8.6369691e-05
  3.2585871e-03  6.2716622e-03  3.0663707e-03 -3.0838398e-03
  7.2643794e-03  1.8298582e-03 -5.0838348e-03  2.8230362e-03
  6.9456697e-03  1.9408650e-03 -2.1707644e-03 -2.6622138e-03
 -4.0640337e-03  8.0507447e-04 -1.8299428e-03  2.4518212e-03
 -3.6292938e-03  1.5122314e-03  3.2507773e-03 -6.4873421e-03
  1.761773

In [18]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

',:RpJTnT.XD..n! Je,z:;MRv!mSEyWVNoWlCGXiz& r3ezHi,-PGOpZBsLtRkttDXhfWzWC\npACDN3;f$oS.sq\n!osJl3xK\n;Ui'

So now we need to create a loss function that can compare that output to the expected output and give us some numeric value representing how close the two were. 

In [19]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

###Compiling the Model
At this point we can think of our problem as a classification problem where the model predicts the probabillity of each unique letter coming next. 


In [20]:
model.compile(optimizer='adam', loss=loss)

###Creating Checkpoints
Now we are going to setup and configure our model to save checkpoinst as it trains. This will allow us to load our model from a checkpoint and continue training it.

In [21]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

###Training
Finally, we will start training the model. 

**If this is taking a while go to Runtime > Change Runtime Type and choose "GPU" under hardware accelerator.**



In [22]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 19s 74ms/step - loss: 2.5291
Epoch 2/50
172/172 [==============================] - 14s 72ms/step - loss: 1.8478
Epoch 3/50
172/172 [==============================] - 14s 73ms/step - loss: 1.6113
Epoch 4/50
172/172 [==============================] - 15s 76ms/step - loss: 1.4848
Epoch 5/50
172/172 [==============================] - 16s 78ms/step - loss: 1.4095
Epoch 6/50
172/172 [==============================] - 15s 78ms/step - loss: 1.3530
Epoch 7/50
172/172 [==============================] - 14s 74ms/step - loss: 1.3070
Epoch 8/50
172/172 [==============================] - 15s 75ms/step - loss: 1.2687
Epoch 9/50
172/172 [==============================] - 15s 77ms/step - loss: 1.2315
Epoch 10/50
172/172 [==============================] - 16s 75ms/step - loss: 1.1948
Epoch 11/50
172/172 [==============================] - 15s 75ms/step - loss: 1.1576
Epoch 12/50
172/172 [==============================] - 15s 76ms/step - loss: 1.1196
E

###Loading the Model
We'll rebuild the model from a checkpoint using a batch_size of 1 so that we can feed one peice of text to the model and have it make a prediction.

In [23]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

Once the model is finished training, we can find the **lastest checkpoint** that stores the models weights using the following line.



In [27]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

###Generating Text
Now we can use the lovely function provided by tensorflow to generate some text using any starting string we'd like.

In [29]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [33]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: Juliet
Juliet what you do contempt her eyes,
To wound thy lord, thy king, thy griefs,
Jest with thy sweet laid into his waste
Here in this chase; I mean, in bed:
You Edward, that in boats befits me;
Made haste your company to blow,
I am anger'd, put us in mind there
So long as if thy cloud knowing
Is, even for the multitude to be do thy death.

BRUTUS:
Come, we'll bring thee to your fortune with
my body to the truth of it. His est thou there?
Stand I am strike.

SICINIUS:
He cannot repossely: but we must
but then?

POMPEY:
Doth your most gracious God!
Murder her to-night.

ROMEO:
Not carr, at December: belike he married well in such a number word
That he disdain should bring which they can see Bastings and truth they lay into the lose,
When they shall know or 'twas more instruction. Alas
This doth not 
